In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import sklearn
import seaborn as sns
from sklearn import metrics
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
%matplotlib inline
style.use("ggplot")

accuracy = metrics.accuracy_score

/Users/kerrylam/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("titanic_training.csv", header = 0) 
test = pd.read_csv("titanic_test.csv", header = 0)
ID = test['PassengerId'] # Extract ID Names From Test Set
data = pd.concat([train, test], axis = 0)
data.shape

(1309, 12)

In [3]:
del data['Name'], data['Ticket'], data['Cabin'], data['PassengerId']

In [4]:
data = pd.get_dummies(data, columns = ['Sex', 'Embarked'], drop_first = True)
data.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,1,0,0,1
1,38.0,71.2833,0,1,1,1.0,0,1,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,0,1
4,35.0,8.0500,0,3,0,0.0,1,0,0,1


In [5]:
del data["Embarked_C"]
data.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,Sex_male,Embarked_Q,Embarked_S
0,22.0,7.2500,0,3,1,0.0,1,0,1
1,38.0,71.2833,0,1,1,1.0,0,0,0
2,26.0,7.9250,0,3,0,1.0,0,0,1
3,35.0,53.1000,0,1,1,1.0,0,0,1
4,35.0,8.0500,0,3,0,0.0,1,0,1


In [6]:
data.fillna(method = 'ffill', inplace = True) # Impute Missing Data
data = data.astype(float) # Convert DF Type to Float

"""Split Data"""
train = data[0:len(train)]
test = data[len(train):]

"""Normalize Data for Faster Computation"""
train = train/train.max().astype(np.float64)
test = test/test.max().astype(np.float64)

In [7]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 5


In [8]:
X = pd.DataFrame.join(train.ix[:, :5], train.ix[:, 6:])
target = train["Survived"]

del test['Survived'] 

In [9]:
print(train.shape)
print(test.shape)

(891, 9)
(418, 8)


In [10]:
train, test_, targ_train, targ_test = train_test_split(X, target, test_size = 0.40, random_state = None)

In [11]:
print ("Train:       ", train.shape)
print ("Train Class: ", targ_train.shape)
print ("Test:        ", test_.shape)
print ("Test Class:  ", targ_test.shape)

Train:        (534, 8)
Train Class:  (534,)
Test:         (357, 8)
Test Class:   (357,)


In [12]:
"""Recombine Training Set With Class"""
train = pd.DataFrame.join(train, targ_train)

In [13]:
train = train.reset_index()
test_ = test_.reset_index()

In [14]:
del train["index"], test_["index"]

### Part 1: 

Partition Training Data into 5 Folds:

- Create train_meta with same rows and IDs as training data
- Create Empty Columns: M1 and M2 (NaNs)

- Create test_meta variable with same rows and IDs as test dataset 
- Create Empty Columns: M1 and M2 (NaNs)

In [15]:
"""Create Train and Test Meta"""
train_meta = pd.DataFrame.copy(train)
train_meta["M1"] = np.nan
train_meta["M2"] = np.nan
print(train.shape)
train_meta.head()

(534, 9)


,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S,Survived,M1,M2
0,0.37500,0.015713,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,NaN,NaN
1,0.50625,0.015127,0.0,1.000000,0.000,1.0,1.0,0.0,0.0,NaN,NaN
2,0.00000,0.015127,0.0,1.000000,0.000,0.0,1.0,0.0,1.0,NaN,NaN
3,0.37500,0.015412,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,NaN,NaN
4,0.42500,0.040989,0.0,0.666667,0.125,1.0,0.0,1.0,0.0,NaN,NaN


In [16]:
test_meta = pd.DataFrame.copy(test_)
# test_meta["M1"] = np.nan
# test_meta["M2"] = np.nan
print(test_.shape)
test_meta.head()

(357, 8)


,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S
0,0.4125,0.018543,0.0,1.000000,0.000,1.0,0.0,1.0
1,0.6375,0.051822,0.0,0.333333,0.000,1.0,0.0,1.0
2,0.5250,0.014932,0.0,1.000000,0.000,1.0,0.0,1.0
3,0.4375,0.020495,0.0,0.666667,0.000,1.0,0.0,1.0
4,0.4125,0.103644,0.0,0.333333,0.125,0.0,0.0,1.0


### Part 2

For each train fold: combine the other 4 into a training set and predict on each fold i 
- train fold 1 = 2,3,4,5 (Predict on fold 1)
- train fold 2 = 1,3,4,5 (Predict on Fold 2)
- train fold 3 = 1,2,4,5 (Predict on Fold 3)
- train fold 4 = 1,2,3,5 (Predict on Fold 4)
- train fold 5 = 1,2,3,4 (Predict on Fold 5)

In [17]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = False)
for train_index, test_index in kf.split(train):
    print("Train:", train_index, "Test:", test_index, "\n")

Train: [107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322
 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340
 341 342 343 344 345 346 347 348 349 350 351

In [18]:
print("Index Location of Target:", train.columns.get_loc("Survived"))

Index Location of Target: 8


In [19]:
"""Copy and Paste Output: Change Target Index In Folds If Necessary"""
j = 1
for train_index, test_index in kf.split(train):
    print ('train_fold{} = train[{}:]'.format(j, 0))
    print ('train_fold{} = train_fold{}.drop(train_fold{}.index[{}:{}])'.format(j, j, j, min(test_index), max(test_index) + 1))
    print ('target_fold{} = train_fold{}.ix[:,8]'.format(j,j))
    print ('test_fold{} = train[{}:{}]'.format(j, min(test_index), max(test_index) + 1), "\n")
    j += 1

train_fold1 = train[0:]
train_fold1 = train_fold1.drop(train_fold1.index[0:107])
target_fold1 = train_fold1.ix[:,8]
test_fold1 = train[0:107] 

train_fold2 = train[0:]
train_fold2 = train_fold2.drop(train_fold2.index[107:214])
target_fold2 = train_fold2.ix[:,8]
test_fold2 = train[107:214] 

train_fold3 = train[0:]
train_fold3 = train_fold3.drop(train_fold3.index[214:321])
target_fold3 = train_fold3.ix[:,8]
test_fold3 = train[214:321] 

train_fold4 = train[0:]
train_fold4 = train_fold4.drop(train_fold4.index[321:428])
target_fold4 = train_fold4.ix[:,8]
test_fold4 = train[321:428] 

train_fold5 = train[0:]
train_fold5 = train_fold5.drop(train_fold5.index[428:534])
target_fold5 = train_fold5.ix[:,8]
test_fold5 = train[428:534] 



In [20]:
train_fold1 = train[0:]
train_fold1 = train_fold1.drop(train_fold1.index[0:107])
target_fold1 = train_fold1.ix[:,8]
test_fold1 = train[0:107] 

train_fold2 = train[0:]
train_fold2 = train_fold2.drop(train_fold2.index[107:214])
target_fold2 = train_fold2.ix[:,8]
test_fold2 = train[107:214] 

train_fold3 = train[0:]
train_fold3 = train_fold3.drop(train_fold3.index[214:321])
target_fold3 = train_fold3.ix[:,8]
test_fold3 = train[214:321] 

train_fold4 = train[0:]
train_fold4 = train_fold4.drop(train_fold4.index[321:428])
target_fold4 = train_fold4.ix[:,8]
test_fold4 = train[321:428] 

train_fold5 = train[0:]
train_fold5 = train_fold5.drop(train_fold5.index[428:534])
target_fold5 = train_fold5.ix[:,8]
test_fold5 = train[428:534] 

In [21]:
print(train_fold1.shape)
print(target_fold1.shape)
print(test_fold1.shape) 

print(train_fold2.shape)
print(target_fold2.shape)
print(test_fold2.shape) 

print(train_fold3.shape)
print(target_fold3.shape)
print(test_fold3.shape) 

print(train_fold4.shape)
print(target_fold4.shape)
print(test_fold4.shape) 

print(train_fold5.shape)
print(target_fold5.shape)
print(test_fold5.shape) 

(427, 9)
(427,)
(107, 9)
(427, 9)
(427,)
(107, 9)
(427, 9)
(427,)
(107, 9)
(427, 9)
(427,)
(107, 9)
(428, 9)
(428,)
(106, 9)


### Part 3
- Use Model M1 to train on each train fold i and predict the excluded fold and store the values back into M1 for its fold ID
- Do the same with M2

In [22]:
"""Instantiate Models"""
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# m2 = RandomForestClassifier()
m2 = KNeighborsClassifier(n_neighbors = 50)
m1 = RandomForestClassifier(n_estimators = 100)
# m1 = LogisticRegression()



In [23]:
"""Train Data On First Model"""
train_folds = [train_fold1, train_fold2, train_fold3, train_fold4, train_fold5]
target_folds = [target_fold1, target_fold2, target_fold3, target_fold4, target_fold5]
test_folds = [test_fold1, test_fold2, test_fold3, test_fold4, test_fold5]

y_pred1 = []
for i, j in zip(train_folds, target_folds):
    m1.fit(i, j)
for k in test_folds:
    target_pred = m1.predict(k)
    y_pred1.append(target_pred) 
    
y_pred1 = np.concatenate(y_pred1, axis=0)
y_pred1 = pd.DataFrame(y_pred1)
train_meta["M1"] = y_pred1

In [24]:
"""Train Data On Second Model"""
y_pred2 = []
for i, j in zip(train_folds, target_folds):
    m2.fit(i, j)
for k in test_folds:
    target_pred2 = m2.predict(k)
    y_pred2.append(target_pred2) 
    
y_pred2 = np.concatenate(y_pred2, axis=0)
y_pred2 = pd.DataFrame(y_pred2)
train_meta["M2"] = y_pred2

### Part 4

Fit m1 Model to train_meta
- Put new predictions into train_meta m1 column
Fit m2 Model to train_meta
- Put new predictions into train_meta m2 column

In [25]:
train_meta

,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S,Survived,M1,M2
0,0.37500,0.015713,0.000000,1.000000,0.000,1.0,0.0,1.0,0.0,0.0,0.0
1,0.50625,0.015127,0.000000,1.000000,0.000,1.0,1.0,0.0,0.0,0.0,0.0
2,0.00000,0.015127,0.000000,1.000000,0.000,0.0,1.0,0.0,1.0,1.0,1.0
3,0.37500,0.015412,0.000000,1.000000,0.000,1.0,0.0,1.0,0.0,0.0,0.0
4,0.42500,0.040989,0.000000,0.666667,0.125,1.0,0.0,1.0,0.0,0.0,0.0
5,0.25000,0.015713,0.000000,1.000000,0.000,1.0,0.0,1.0,0.0,0.0,0.0
6,0.45000,0.051505,0.000000,0.333333,0.000,1.0,0.0,1.0,1.0,1.0,1.0
7,0.22500,0.015330,0.166667,1.000000,0.125,1.0,0.0,1.0,0.0,0.0,0.0
8,0.46250,0.057971,0.166667,0.333333,0.000,1.0,0.0,0.0,0.0,0.0,0.0
9,0.42500,0.025374,0.000000,0.666667,0.000,1.0,0.0,1.0,0.0,0.0,0.0


### Part 5
- Predict m1 on test_meta
- Predict m2 on test_meta

Part 4
Use another model (S) and use train_meta to make final predictions on test_meta 

In [26]:
train_meta.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S,Survived,M1,M2
0,0.37500,0.015713,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,0.0,0.0
1,0.50625,0.015127,0.0,1.000000,0.000,1.0,1.0,0.0,0.0,0.0,0.0
2,0.00000,0.015127,0.0,1.000000,0.000,0.0,1.0,0.0,1.0,1.0,1.0
3,0.37500,0.015412,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,0.0,0.0
4,0.42500,0.040989,0.0,0.666667,0.125,1.0,0.0,1.0,0.0,0.0,0.0


### Store New Targets Elsewhere For Training And Delete Them Off Training Set

In [27]:
train_meta_Class = train_meta["Survived"]
train_meta_M1 = train_meta["M1"]
train_meta_M2 = train_meta["M2"]

del train_meta["Survived"]

training = pd.DataFrame.copy(train_meta)

del train_meta["M1"], train_meta["M2"]

In [28]:
train_meta.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S
0,0.37500,0.015713,0.0,1.000000,0.000,1.0,0.0,1.0
1,0.50625,0.015127,0.0,1.000000,0.000,1.0,1.0,0.0
2,0.00000,0.015127,0.0,1.000000,0.000,0.0,1.0,0.0
3,0.37500,0.015412,0.0,1.000000,0.000,1.0,0.0,1.0
4,0.42500,0.040989,0.0,0.666667,0.125,1.0,0.0,1.0


In [29]:
"""Train Data On Third Model"""
from sklearn.linear_model import LogisticRegression
m1.fit(train_meta, train_meta_M1)
target_pred_M1 = m1.predict(test_meta)

m2.fit(train_meta, train_meta_M2)
target_pred_M2 = m2.predict(test_meta)

test_meta["M1"] = target_pred_M1
test_meta["M2"] = target_pred_M2

In [30]:
training.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.37500,0.015713,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,0.0
1,0.50625,0.015127,0.0,1.000000,0.000,1.0,1.0,0.0,0.0,0.0
2,0.00000,0.015127,0.0,1.000000,0.000,0.0,1.0,0.0,1.0,1.0
3,0.37500,0.015412,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,0.0
4,0.42500,0.040989,0.0,0.666667,0.125,1.0,0.0,1.0,0.0,0.0


In [31]:
test_meta.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_male,Embarked_Q,Embarked_S,M1,M2
0,0.4125,0.018543,0.0,1.000000,0.000,1.0,0.0,1.0,1.0,0.0
1,0.6375,0.051822,0.0,0.333333,0.000,1.0,0.0,1.0,1.0,0.0
2,0.5250,0.014932,0.0,1.000000,0.000,1.0,0.0,1.0,0.0,0.0
3,0.4375,0.020495,0.0,0.666667,0.000,1.0,0.0,1.0,0.0,0.0
4,0.4125,0.103644,0.0,0.333333,0.125,0.0,0.0,1.0,1.0,1.0


### Part 6 Predict On Actual Testing Set

In [32]:
s = LogisticRegression()
s.fit(training, train_meta_Class)
target_pred_Class = s.predict(test_meta)

test_meta["Survived"] = target_pred_Class

In [33]:
print(round(accuracy(targ_test, target_pred_Class)*100, 2), "% Accuracy")

80.11 % Accuracy
